In [148]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from scipy import sparse

In [149]:
def convert_labels(y, C):
  Y = sparse.coo_matrix((np.ones_like(y),
  (y, np.arange(len(y)))), shape = (C, len(y))).toarray()
  return Y

def softmax_stable(Z):
  e_Z = np.exp(Z - np.max(Z, axis = 0, keepdims = True))
  A = e_Z / e_Z.sum(axis = 0)
  return A

def softmax(Z):
  e_Z = np.exp(Z)
  A = e_Z / e_Z.sum(axis = 0)
  return A

def softmax_regression(X, y, W_init, eta, tol = 1e-4, max_count = 10000):
  W = [W_init]
  C = W_init.shape[1]
  Y = convert_labels(y, C)
  it = 0
  N = X.shape[1]
  d = X.shape[0]

  count = 0
  check_w_after = 20
  while count < max_count:
# mix data
    mix_id = np.random.permutation(N)
    for i in mix_id:
      xi = X[:, i].reshape(d, 1)
      yi = Y[:, i].reshape(C, 1)
      ai = softmax(np.dot(W[-1].T, xi))
      W_new = W[-1] + eta*xi.dot((yi - ai).T)
      count += 1
      # stopping criteria
      if count%check_w_after == 0:
        if np.linalg.norm(W_new - W[-check_w_after]) < tol:
          return W

      W.append(W_new)
  return W
  
def cost(X, Y, W):
  A = softmax(W.T.dot(X))
  return -np.sum(Y*np.log(A))

def pred(W, X):
  A = softmax_stable(W.T.dot(X))
  return np.argmax(A, axis = 0)

In [150]:
iris = load_iris()
X = iris.data
y = iris.target


# Logistic Regression

In [151]:
flo_1X, flo_2X, flo_3X = X[:50, :], X[50:100, :], X[100:150, :]
flo_1y, flo_2y, flo_3y = y[:50], y[50:100], y[100:150]

idx1 = np.random.permutation(len(flo_1X))
idx2 = np.random.permutation(len(flo_2X))
idx3 = np.random.permutation(len(flo_3X))

flo_1X, flo_1y = flo_1X[idx1], flo_1y[idx1]
flo_2X, flo_2y = flo_2X[idx2], flo_2y[idx2]
flo_3X, flo_3y = flo_3X[idx3], flo_3y[idx3]

X_train = np.concatenate((flo_1X[:40], flo_2X[:40], flo_3X[:40])).T
X_test  = np.concatenate((flo_1X[40:], flo_2X[40:], flo_3X[40:])).T
y_train = np.concatenate((flo_1y[:40], flo_2y[:40], flo_3y[:40]))
y_test  = np.concatenate((flo_1y[40:], flo_2y[40:], flo_3y[40:]))

print(X_train.shape)  
print(y_train.shape)  


(4, 120)
(120,)


In [152]:
eta = 0.05
C = 3
d = X_train.shape[0]
W_init = np.random.randn(d, C)
W = softmax_regression(X_train, y_train, W_init, eta)
print(W[-1].shape)
print(X_test.shape)

(4, 3)
(4, 30)


In [153]:
y_pred = pred(W[-1], X_test)
print(y_pred)


[0 0 0 0 0 0 0 0 0 0 2 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2]


In [154]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

print(f'accuracy:{accuracy_score(y_test, y_pred)}')
print(f'precision:{precision_score(y_test, y_pred,average='macro')}')
print(f'recall:{recall_score(y_test, y_pred,average='macro')}')

accuracy:0.9666666666666667
precision:0.9696969696969697
recall:0.9666666666666667


# Naive Bayes

In [155]:
from scipy.stats import norm
def predict_NB_gaussian_class(X, mu_list, std_list, pi_list):
  scores_list = []
  classes = len(mu_list)
  for c in range(classes):
    score = norm.pdf(x = X[0], loc = mu_list[c][0][0], scale = std_list[c][0][0]) * norm.pdf(x = X[1], loc = mu_list[c][0][1], scale = std_list[c][0][1]) * norm.pdf(x = X[2], loc = mu_list[c][0][2], scale = std_list[c][0][2]) * norm.pdf(x = X[3], loc = mu_list[c][0][3], scale = std_list[c][0][3]) * pi_list[c]
    scores_list.append(score)
  return np.argmax(scores_list)

In [156]:
X_train, X_test = X_train.T, X_test.T

In [157]:
classes = np.unique(y_train)
mu_list = [X_train[y_train == c].mean(axis=0, keepdims=True) for c in classes]
std_list = [X_train[y_train == c].std(axis=0, keepdims=True) for c in classes]
pi_list = np.array([np.mean(y == c) for c in classes])

In [158]:
y_pred = np.zeros(X_test.shape[0])
for i in range(X_test.shape[0]):
  y_pred[i] = predict_NB_gaussian_class(X_test[i, :], mu_list, std_list, pi_list).astype(int)


In [159]:
print(f'accuracy:{accuracy_score(y_test, y_pred)}')
print(f'precision:{precision_score(y_test, y_pred,average='macro')}')
print(f'recall:{recall_score(y_test, y_pred,average='macro')}')

accuracy:0.9333333333333333
precision:0.9444444444444445
recall:0.9333333333333332
